<a href="https://colab.research.google.com/github/bhgtankita/DL_Colab/blob/master/Deep_Residual_Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deep Residual UNet

![link text](https://github.com/nikhilroxtomar/Deep-Residual-Unet/raw/master/images/arch.png)

![link text](https://github.com/nikhilroxtomar/Deep-Residual-Unet/raw/5bc7ba987856f98bc49514ea574e51b4fce75c4e/images/basic%20block.png)

# Implementation with Neucleus segmentation problem

In [0]:
import sys
import os
import random

import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

import pandas as pd

#seeding
seed = 2019
random.seed = seed
np.random.seed = seed
tf.seed = seed

## If you set the np.random.seed(a_fixed_number) every time you call the numpy's other random function, the result will be the same:

In [0]:
class DataGen(keras.utils.Sequence):
  def __init__(self, ids, path, batch_size=8, image_size=128):
    self.ids = ids
    self.path = path
    self.batch_size = batch_size
    self.image_size = image_size
    self.on_epoch_end()
    
  def __load__(self, id_name):
    #paths
    image_path = os.path.join(self.path, "images", id_name) + ".png"
    mask_path = os.path.join(self.path, "masks", id_name) + ".png"
    
    # reading images
    image = cv2.imread(image_path)
    image = cv2.resize(image, (self.image_size, self.image_size))

    # reading mask
    mask = cv2.imread(mask_path, 0)
    mask = cv2.resize(mask, (self.image_size, self.image_size))    
    mask = np.expand_dims(mask, axis=-1)
    
    #normalizing
    image = image/255.0
    mask = mask/255.0
    
    return image, mask
  
  def __getitem__(self, index):
    if (index+1)*self.batch_size > len(self.ids):
      self.batch_size = len(self.ids) - index*self.batch_size
      
    files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
    
    image = []
    mask = []
    
    for id_name in files_batch:
      _img, _mask = self.__load__(id_name)
      image.append(_img)
      mask.append(_mask)
      
    image = np.array(image)
    mask = np.array(mask)
    
    return image, mask
  
  def on_epoch_end(self):
    pass
  
  def __len__(self):
    return int(np.ceil(len(self.ids)/float(self.batch_size)))
    

# Upload data from Kaggle

In [14]:
#allows us to upload files into colab
#we'll need to upload the kaggle.json file
#in kaggle, under accounts, click 'create new API token'
#upload the kaggle.json file that is automatically downloaded
from google.colab import files
files.upload()

{}

In [15]:
#ensure its there
!ls -lha kaggle.json

ls: cannot access 'kaggle.json': No such file or directory


In [16]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [17]:
#lets now download our dataset
!kaggle competitions download -c tgs-salt-identification-challenge

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 6, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [18]:
!ls

sample_data


In [19]:
!unzip -q train.zip

unzip:  cannot find or open train.zip, train.zip.zip or train.zip.ZIP.


In [20]:
!ls

sample_data


# Hyperparameters

In [21]:
train_csv = pd.read_csv("train.csv")
train_ids = train_csv["id"].values

image_size = 128
batch_size = 16

val_data_size = 200

valid_ids = train_ids[:val_data_size]
train_ids = train_ids[val_data_size:]

FileNotFoundError: ignored

In [0]:
!mkdir dataset

In [0]:
cd dataset

In [0]:
!mkdir train

In [0]:
cd train

In [0]:
!ls

In [0]:
!mkdir images

In [0]:
!mkdir masks

In [0]:
cd ..

In [0]:
!mv -v /content/images/* /content/dataset/train/images/

In [0]:
!mv -v /content/masks/* /content/dataset/train/masks/

#Paths

In [0]:
dataset_path = "dataset/"
train_path = os.path.join(dataset_path, "train/")

In [0]:
gen = DataGen(train_ids, train_path, batch_size=batch_size, image_size=image_size)
x, y = gen.__getitem__(0)
print(x.shape, y.shape)

In [0]:
r = random.randint(0, len(x)-1)

fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
ax = fig.add_subplot(1, 2, 1)
ax.imshow(x[r])
ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(y[r], (image_size, image_size)), cmap="gray")

# Network Architecture

In [0]:
def bn_act(x, act=True):
  x = keras.layers.BatchNormalization()(x)
  if act == True:
    x = keras.layers.Activation("relu")(x)
  return x

def conv_block(x, filters, kernel_size=(3,3), padding="same", strides=1):
  conv = bn_act(x)
  conv = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
  return conv

def stem(x, filters, kernel_size=(3,3), padding="same", strides=1):
  conv = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
  conv = conv_block(conv, filters, kernel_size=kernel_size, padding=padding, strides=strides)
  
  shortcut = keras.layers.Conv2D(filters, kernel_size=(1,1), padding=padding, strides=strides)(x)
  shortcut = bn_act(shortcut, act=False)
  
  output = keras.layers.Add()([conv, shortcut])
  return output

def residual_block(x, filters, kernel_size=(3,3), padding="same", strides=1):
  res = conv_block(x, filters, kernel_size=kernel_size, padding=padding, strides=strides)
  res = conv_block(res, filters, kernel_size=kernel_size, padding=padding, strides=1)
  
  shortcut = keras.layers.Conv2D(filters, kernel_size=(1,1), padding=padding, strides=strides)(x)
  shortcut = bn_act(shortcut, act=False)
  
  output = keras.layers.Add()([shortcut, res])
  return output

def upsample_concat_block(x, xskip):
  u = keras.layers.UpSampling2D((2,2))(x)
  c = keras.layers.Concatenate()([u, xskip])
  return c

# UNET Model

In [0]:
def ResUNet():
    f = [16, 32, 64, 128, 256]
    inputs = keras.layers.Input((image_size, image_size, 3))
    
    ## Encoder
    e0 = inputs
    e1 = stem(e0, f[0])
    e2 = residual_block(e1, f[1], strides=2)
    e3 = residual_block(e2, f[2], strides=2)
    e4 = residual_block(e3, f[3], strides=2)
    e5 = residual_block(e4, f[4], strides=2)
    
    ## Bridge
    b0 = conv_block(e5, f[4], strides=1)
    b1 = conv_block(b0, f[4], strides=1)
    
    ## Decoder
    u1 = upsample_concat_block(b1, e4)
    d1 = residual_block(u1, f[4])
    
    u2 = upsample_concat_block(d1, e3)
    d2 = residual_block(u2, f[3])
    
    u3 = upsample_concat_block(d2, e2)
    d3 = residual_block(u3, f[2])
    
    u4 = upsample_concat_block(d3, e1)
    d4 = residual_block(u4, f[1])
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(d4)
    model = keras.models.Model(inputs, outputs)
    return model

#Loss Function

In [0]:
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = tf.layers.flatten(y_true)
    y_pred_f = tf.layers.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [0]:
model = ResUNet()
adam = keras.optimizers.Adam()
model.compile(optimizer=adam, loss=dice_coef_loss, metrics=[dice_coef])
model.summary()

# Training

In [0]:
train_gen = DataGen(train_ids, train_path, batch_size=batch_size, image_size=image_size)
valid_gen = DataGen(valid_ids, train_path, batch_size=batch_size, image_size=image_size)

train_steps = len(train_ids)//batch_size
valid_steps = len(valid_ids)//batch_size

epochs = 200

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=train_steps, validation_steps=valid_steps, epochs=epochs)

#Testing

In [0]:
model.save_weights("ResUNet.h5")

In [0]:
print("\n      Ground Truth            Predicted Value")

for i in range(1, 5, 1):
    ## Dataset for prediction
    x, y = valid_gen.__getitem__(i)
    result = model.predict(x)
    result = result > 0.4
    
    for i in range(len(result)):
        fig = plt.figure()
        fig.subplots_adjust(hspace=0.4, wspace=0.4)

        ax = fig.add_subplot(1, 2, 1)
        ax.imshow(np.reshape(y[i]*255, (image_size, image_size)), cmap="gray")

        ax = fig.add_subplot(1, 2, 2)
        ax.imshow(np.reshape(result[i]*255, (image_size, image_size)), cmap="gray")

Reference: https://github.com/nikhilroxtomar/Deep-Residual-Unet/blob/master/Deep%20Residual%20UNet.ipynb

Video: https://www.youtube.com/watch?v=BOoBWRTpaKk

Dataset: https://www.kaggle.com/c/tgs-salt-identification-challenge